This notebook shows the development of an Artificial Neural Network with focus on classification.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
import sklearn.metrics as metrics
import sklearn.datasets as sk_datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [2]:
class Function:
	
	#
	# Sigmoid function for the activation
	# of a neuron, where h is the dot
	# product of X (input) and theta (weights)
	#
	def act_Sigmoid(h):
		sig = 1. / (1. + np.exp(-h))
		return sig

	#
	# Derivative of the sigmoid function. It
	# is used as part of the backpropagation
	# algorithm
	#
	def act_Sigmoid_derivative(h):
		sig = act_Sigmoid(h)
		derivative = sig*(1-sig)
		return derivative

	#
	# Hyberbolic tangent function for the
	# acivation of a neuron, where h is the
	# dot product of X (input) and theta (weights)
	#	
	def act_Tanh(h):
		tanh = (2 / (1+np.exp(-2*h)))-1

	#
	# Deivative of the hyperbolic tangent
	# function. It is used as part of the	
	# back propagation algorithm.
	#
	def act_Tanh_derivative(h):
		tanh_l = (4*np.exp(-2*h))/((1+np.exp(-2*h))**2)
		return tanh_l

	#
	# Cross entropy loss function, where h
	# is the activation of the last layer.
	# It computes the error of the predicted
	# class and the correct one.
	#
	def err_CrossEntropy(predY,y):
		eps = np.finfo(np.float128).eps
		predY[predY < eps] = eps
		predY[predY > 1.-eps] = 1.-eps
		return -np.multiply(np.log(predY),y) - np.multiply((np.log(1-predY)),(1-y))

	#
	# Derivative of the SMD loss function.
	# It is used in the back propagation
	# algorithm.
	#
	def err_CrossEntropy_derivative(X,predY,y):
		error = (predY - y)
		grad = np.dot(X.transpose(),error)
		return grad

	#
	# Sum of the squared differences (SMD) loss
	# function, where h is the activation of the
	# last layer. It computes the error of the
	# predicted class and the correct one.
	#
	def err_SMD(predY,y):
		error = np.square((predY - y)).sum()
		return error/2

	#
	# Sum of the squared differences (SMD) loss
	# function, where h is the activation of the
	# last layer. It computes the error of the
	# predicted class and the correct one.
	#
	def err_SMD_derivative(X,predY,y):
		error = (predY - y)
		grad = np.dot(X.transpose(),error)
		return grad

In [11]:
class ANN:

	def __init__(self, activation):
		self.act_func = Function.__dict__[activation]
		self.act_derivative = Function.__dict__[activation+"_derivative"]

	def initialize_random_weights(self, n_input, n_perceptron, n_classes):
		self.n_hidden_layers = len(n_perceptron)

		self.hidden_layers = []
		self.activation = []

		for l in range(self.n_hidden_layers):
			if (l == 0):
				w = np.random.rand(n_input+1,n_perceptron[0])
			else:
				w = np.random.rand(n_perceptron[l-1]+1,n_perceptron[l])
			self.hidden_layers.append(w)

		if (self.n_hidden_layers == 0):
			self.output_layer = np.random.rand(n_input+1,n_classes)
		else:
			self.output_layer = np.random.rand(n_perceptron[self.n_hidden_layers-1]+1,n_classes)


	def initialize_fixed_weights(self, w):
		self.hidden_layers = w[:-1]
		self.output_layer = w[-1]
		self.n_hidden_layers = len(w)-1


	def show_weights(self):

		for l in range(self.n_hidden_layers):
			print("Hidden Layer ",str(l+1))
			print(self.hidden_layers[l],"\n")

		print("Output Layer ")
		print(self.output_layer,"\n")

	def show_setup(self):
				
		print("--- Input size: ",str(self.hidden_layers[0].shape[0]-1))
		print("--- Number of hidden layers: ",str(self.n_hidden_layers))
		print("--- Number of perceptrons at each layer: ")
		for l in range(self.n_hidden_layers):
			print("------ HL "+str(l+1)+": "+str(self.hidden_layers[l].shape[1]))
		print("--- Number of classes: "+str(self.output_layer.shape[1]),"\n")

	def foward_propagation(self, X):

		inp = np.insert(X,0,1,axis=1)

		for l in range(self.n_hidden_layers):
			out = np.matmul(inp, self.hidden_layers[l])
			sig = self.act_func(out)
			inp = np.insert(sig,0,1,axis=1)

		out = np.matmul(inp, self.output_layer)
		sig = self.act_func(out)

		return out,sig


In [13]:
#----------------------------------
#        ANN Initialization
#----------------------------------

# Random Init
teste = ANN("act_Sigmoid")
teste.initialize_random_weights(2, [2], 2)
teste.show_weights()
teste.show_setup()

'''
# Fixed Init
w1 = np.array([[1,2],[3,4],[5,6]])
w2 = np.array([[7,8],[9,10],[11,12]])
w = [w1,w2]

teste = ANN("act_Sigmoid")
teste.initialize_fixed_weights(w)
teste.show_weights()
teste.show_setup()
'''

#----------------------------------
#           Toy Examples
#----------------------------------

X = np.array([[1,2],[3,4]])
Y, aY = teste.foward_propagation(X)
print("Y\n",Y)
print("\na(Y)\n",aY)

Hidden Layer  1
[[0.08932856 0.03602717]
 [0.64642563 0.997865  ]
 [0.05771276 0.77755773]] 

Output Layer 
[[0.92346771 0.09930999]
 [0.27451155 0.74895807]
 [0.97537742 0.42216732]] 

--- Input size:  2
--- Number of hidden layers:  1
--- Number of perceptrons at each layer: 
------ HL 1: 2
--- Number of classes: 2 

Y
 [[2.02309746 1.01686996]
 [2.14530756 1.1987233 ]]

a(Y)
 [[0.88320091 0.73436246]
 [0.89522947 0.76829759]]
